# Workflow nets test

This notebook is a test to redisign the workflow of gepard using three main components:
- Model
- Theory
- Fitter 

The idea is to feed a neural network $NN$ with several datasets containing $(xB,t,Q2,val,stat_error,syst_error)$ using just $(xB,t,Q2)$ as inputs. The network is trained to predict the comptom form factors $(ImH, ReH, ImE, ReE, ImHt, ReHt, ImEt, ReEt)$ and the D-term $D(t)$. We will start by predicting two outputs. These outputs are then passed to a theory module which computes the observables values (CS, BSA, etc.).

The predicted observables are compared to the input values via loss function $L$. The $NN$ is optimized by minimising the loss.

The followin cells provide a walkthrough of how this idea is implemented in code. 


## First we need to import all the necessary modules

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import gepard as g
import gepard.plots as gplot

In [ ]:
import torch
import numpy as np 
#np.set_printoptions(legacy='1.25')
import matplotlib
import matplotlib.pyplot as plt
import shelve, logging, copy
logging.basicConfig(level=logging.ERROR) 
import pandas as pd 
from scipy.integrate import quad
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm

from math import sqrt
from typing import List, Union
import time

In [ ]:
# Define file path
DIR = '/Users/higuera-admin/Documents/Programs/ldrdgff/Analysis/Results'

## Datasets

In [ ]:
sys.path.append('/Users/higuera-admin/Documents/Programs/ldrdgff/gepard/src/gepard/')

import mydatafiles
from mydatafiles import ep2epgamma

mydset = g.data.loaddata(mydatafiles)
mydset.update(g.data.loaddata(ep2epgamma))

In [ ]:
fitpoints = mydset[182] + mydset[192] + g.dset[101] + g.dset[102] + g.dset[8]
g.describe_data(fitpoints)

## Theory Module

Here is the theory module which creates the predicted data based on the neural network predictions.